# Transformación de los datos 

In [1]:
# !pip install pyarrow

In [5]:
import pandas as pd
import os
import warnings

## Unión de los datos

In [6]:
anio_arr = pd.read_csv('./catalogos_api/Anio_Agricola.csv').ID.to_list()
cultivo_arr = pd.read_csv('./catalogos_api/Cultivo.csv').ID.to_list()
mes_arr = pd.read_csv('./catalogos_api/Mes_Agricola.csv').ID.to_list()
cultivo_arr.remove(0) # 0: Todos

In [7]:
# anio,mes,ciclo,cultivo, entidad, modalidad
columnas = ["Entidad", "Municipio", "Superficie Sembrada", "Superficie Cosechada", "Superficie Siniestrada", "Produccion", "Rendimiento","Anio", "Mes", "Cultivo"]
df_raw = pd.DataFrame(columns=columnas)

total = 0
for anio in anio_arr:
    for i in range(1,13):
        df_aux = pd.read_csv("./raw/"+str(anio)+"/Avance_Agricola_"+str(anio)+ "_" + str(i) + '.csv')
        df_raw = pd.concat([df_raw, df_aux], axis=0)
        total += df_aux.shape[0]

df_raw['Superficie Siniestrada'] = pd.to_numeric(df_raw['Superficie Siniestrada'], errors='coerce')

C:\Users\vesna\AppData\Local\Temp\ipykernel_12924\2729179447.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_raw = pd.concat([df_raw, df_aux], axis=0)


In [8]:
if not os.path.exists('raw/Avance_Agricola_Historico.parquet'):
    name_csv = "Avance_Agricola_Historico"
    ruta_completa = os.path.join("raw",name_csv)
    df_raw.to_parquet(ruta_completa + '.parquet', index=False)

In [9]:
print("Total de datos: ",total)

Total de datos:  992213


## Decodificación de variables

In [10]:
df = pd.read_parquet('./raw/Avance_Agricola_Historico.parquet')

In [11]:
catalogo_cultivos = pd.read_csv('./catalogos_api/Cultivo.csv')
catalogo_mes = pd.read_csv('./catalogos_api/Mes_Agricola.csv')

catalogo_cultivos = catalogo_cultivos.set_index('ID')['Name'].to_dict()
catalogo_mes = catalogo_mes.set_index('ID')['Name'].to_dict()

df['Cultivo'] = df['Cultivo'].map(catalogo_cultivos)
df['Mes'] = df['Mes'].map(catalogo_mes)


In [12]:
df.to_parquet('tidy.parquet', index=False)

In [13]:
df = pd.read_parquet('tidy.parquet')
df

,Entidad,Municipio,Superficie Sembrada,Superficie Cosechada,Superficie Siniestrada,Produccion,Rendimiento,Anio,Mes,Cultivo
0,Aguascalientes,Aguascalientes,8.00,0.00,0.0,0.00,0,2023,Enero,Agave
1,Baja California Sur,La Paz,3.00,0.00,0.0,0.00,0,2023,Enero,Agave
2,Colima,Manzanillo,86.29,0.00,0.0,0.00,0,2023,Enero,Agave
3,Durango,Durango,13.00,0.00,0.0,0.00,0,2023,Enero,Agave
4,Durango,Mezquital,110.00,0.00,0.0,0.00,0,2023,Enero,Agave
...,...,...,...,...,...,...,...,...,...,...
992208,Puebla,Domingo Arenas,12.00,12.00,0.0,110.00,9.17,2018,Diciembre,Zarzamora
992209,Puebla,Huejotzingo,10.00,10.00,0.0,98.00,9.80,2018,Diciembre,Zarzamora
992210,Puebla,Tochimilco,1.00,1.00,0.0,4.10,4.10,2018,Diciembre,Zarzamora
992211,Querétaro,San Juan del Río,3.00,3.00,0.0,23.10,7.70,2018,Diciembre,Zarzamora
